In [1]:
import yfinance as yf
import pandas as pd

# Step 1: Data Collection
symbol = 'AAPL'
data = yf.download(symbol, start='2020-01-01', end='2023-01-01')
data.reset_index(inplace=True)

# Output the first few rows of the data
print("Step 1: Data Collection")
print(data.head())  # Show first few rows of the data


[*********************100%***********************]  1 of 1 completed

Step 1: Data Collection
        Date       Open       High        Low      Close  Adj Close     Volume
0 2020-01-02  74.059998  75.150002  73.797501  75.087502  72.876122  135480400
1 2020-01-03  74.287498  75.144997  74.125000  74.357498  72.167603  146322800
2 2020-01-06  73.447502  74.989998  73.187500  74.949997  72.742661  118387200
3 2020-01-07  74.959999  75.224998  74.370003  74.597504  72.400536  108872000
4 2020-01-08  74.290001  76.110001  74.290001  75.797501  73.565193  132079200


In [3]:
import numpy as np  # Import NumPy for calculations

# Step 2: Calculate Indicators
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# Calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = calculate_rsi(data)

# Output the last 60 rows including new indicators
print("\nStep 2: Calculate Indicators")
print(data[['Close', 'SMA50', 'SMA200', 'RSI']].tail(60))  # Show last 60 rows



Step 2: Calculate Indicators
          Close       SMA50     SMA200        RSI
696  145.429993  159.033199  159.30130  42.826021
697  140.089996  158.687999  159.13680  31.209191
698  140.419998  158.246199  158.96070  27.237573
699  138.979996  157.795599  158.77420  28.612881
700  138.339996  157.362199  158.56425  28.898000
701  142.990005  156.899399  158.38275  39.797057
702  138.380005  156.350800  158.17775  34.790080
703  142.410004  155.892000  157.99880  39.319180
704  143.750000  155.469600  157.82970  42.949762
705  143.860001  155.048400  157.63895  51.919873
706  143.389999  154.531400  157.45740  58.076567
707  147.270004  154.107000  157.31915  57.588170
708  149.449997  153.653999  157.20640  55.529864
709  152.339996  153.236999  157.10725  59.032849
710  149.350006  152.763399  156.99305  55.616066
711  144.800003  152.168399  156.84165  56.904143
712  155.740005  151.800200  156.74270  67.128809
713  153.339996  151.436599  156.64845  65.718036
714  150.649994  151

In [5]:
# Step 3: Create Trading Signals
data['Signal'] = 0

# Define buy conditions
buy_condition = (data['SMA50'] > data['SMA200']) & (data['RSI'] < 30)

# Create signals based on the conditions
data['Signal'] = np.where(buy_condition, 1, 0)
data['Position'] = data['Signal'].diff()

# Output the last few rows to check signals
print("\nStep 3: Create Trading Signals")
print(data[['Close', 'SMA50', 'SMA200', 'RSI', 'Signal', 'Position']].tail(60))  # Show last 60 rows



Step 3: Create Trading Signals
          Close       SMA50     SMA200        RSI  Signal  Position
696  145.429993  159.033199  159.30130  42.826021       0       0.0
697  140.089996  158.687999  159.13680  31.209191       0       0.0
698  140.419998  158.246199  158.96070  27.237573       0       0.0
699  138.979996  157.795599  158.77420  28.612881       0       0.0
700  138.339996  157.362199  158.56425  28.898000       0       0.0
701  142.990005  156.899399  158.38275  39.797057       0       0.0
702  138.380005  156.350800  158.17775  34.790080       0       0.0
703  142.410004  155.892000  157.99880  39.319180       0       0.0
704  143.750000  155.469600  157.82970  42.949762       0       0.0
705  143.860001  155.048400  157.63895  51.919873       0       0.0
706  143.389999  154.531400  157.45740  58.076567       0       0.0
707  147.270004  154.107000  157.31915  57.588170       0       0.0
708  149.449997  153.653999  157.20640  55.529864       0       0.0
709  152.339996 

In [7]:
# Step 4: Backtesting Implementation
initial_capital = 10000
shares = 0
portfolio_value = []

# Check for 'Position' column existence
print("\nChecking for 'Position' column...")
if 'Position' not in data.columns:
    print("Error: 'Position' column not found. Check the signal creation step.")
else:
    print("'Position' column found. Proceeding with backtesting.")
    
    # Simulate trading
    for index, row in data.iterrows():
        print(f"Row index: {index}, Position: {row['Position']}")  # Debugging output
        
        if row['Position'] == 1:  # Buy signal
            shares = initial_capital // row['Close']
            initial_capital -= shares * row['Close']
        elif row['Position'] == -1:  # Sell signal
            if shares > 0:  # Check if we have shares to sell
                initial_capital += shares * row['Close']
                shares = 0

        # Calculate total portfolio value
        portfolio_value.append(initial_capital + shares * row['Close'])

    # Store the portfolio value in the DataFrame
    data['Portfolio Value'] = portfolio_value

    # Output final portfolio value
    print("\nStep 4: Backtesting Implementation")
    print(data[['Date', 'Portfolio Value']].tail())  # Show last few rows of portfolio value



Checking for 'Position' column...
'Position' column found. Proceeding with backtesting.
Row index: 0, Position: nan
Row index: 1, Position: 0.0
Row index: 2, Position: 0.0
Row index: 3, Position: 0.0
Row index: 4, Position: 0.0
Row index: 5, Position: 0.0
Row index: 6, Position: 0.0
Row index: 7, Position: 0.0
Row index: 8, Position: 0.0
Row index: 9, Position: 0.0
Row index: 10, Position: 0.0
Row index: 11, Position: 0.0
Row index: 12, Position: 0.0
Row index: 13, Position: 0.0
Row index: 14, Position: 0.0
Row index: 15, Position: 0.0
Row index: 16, Position: 0.0
Row index: 17, Position: 0.0
Row index: 18, Position: 0.0
Row index: 19, Position: 0.0
Row index: 20, Position: 0.0
Row index: 21, Position: 0.0
Row index: 22, Position: 0.0
Row index: 23, Position: 0.0
Row index: 24, Position: 0.0
Row index: 25, Position: 0.0
Row index: 26, Position: 0.0
Row index: 27, Position: 0.0
Row index: 28, Position: 0.0
Row index: 29, Position: 0.0
Row index: 30, Position: 0.0
Row index: 31, Positio